In [1]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install transformers
!pip install language_tool_python
!pip install moviepy librosa
!pip install SpeechRecognition
! pip install librosa


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-briaof3b
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-briaof3b
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.0 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802825 sha256=f88fd80504158661b6e3599493ac4a3fdf345157c4c42572bde6311d63d662cc
  Stored in directory: /tmp/pip-ephem-wheel-cache-n72uipnx/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu22

In [ ]:
voice_data =[]
text_data = []

In [ ]:
def transcrib(path):
  import whisper

  model = whisper.load_model("medium.en")
  transcribed_text = model.transcribe(path, language="en", fp16=False)
  transcribed_text = transcribed_text["text"]
  return transcribed_text

In [ ]:
transcribed_text = " Hello, my is name had Paul Bernal. I was currently live in Los Angeles, California in the United States of America. I'm 23 years old and I graduated from high school in 2008 from Londora High School. Londora is the city where I grew up. Londora is approximately 30 minutes away from Los Angeles and only 45 minutes from the beach. I graduated from high school with a 3.0 GPA. While I was in high school I was also a marching man. I played many instruments including the trombone, trumpet, drums, and guitar. After high school I continued my education at Citrus College which is also located in Londora, California. At Citrus College I obtained two associates degrees. I obtained one degree in Administration of Justice or Criminal Justice and another in Biological, Physical Sciences, and Mathematics. After college I got a job part time with the City of Ontario as a police cadet. As a police cadet I was able to help many citizens in the City of Ontario with police related problems. After becoming a police cadet I went on to become a Sheriff's Deputy with the San Bernardino County Sheriff's Department. In order for me to become a Sheriff's Deputy I had to graduate a six month long academy which was very tough. It required a lot of hard work. After I became a Sheriff's Deputy I decided I wanted to do something different so I became a service agent for a car rental company called Enterprise where I work today. My goal for this video is to be able to show you that I have a strong interest in teaching. Not only do I want to be able to travel the world but I want to be able to use my education and knowledge of the English language that I have learned since I was a child to other students who want to learn. So if you are able to consider me for this position I greatly appreciate it. Again my name is Paul Bernal and I appreciate your time. Thank you."

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import re

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("textattack/distilbert-base-uncased-CoLA")
model = AutoModelForSequenceClassification.from_pretrained("textattack/distilbert-base-uncased-CoLA")

def is_grammatically_correct(sentence):
    # Tokenize the sentence
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)

    # Get model's prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)  # 0 means unacceptable, 1 means acceptable

    return predictions[0].item()

def check_paragraph_grammar(paragraph):
    # Split the paragraph into sentences
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', paragraph)

    total_sentences = len(sentences)
    error_count = 0

    for sentence in sentences:
        result = is_grammatically_correct(sentence)
        if result == 0:  # If the sentence is not grammatically correct
            error_count += 1
            print(f"Error found in sentence: {sentence}")

    error_rate = (error_count / total_sentences) * 100
    return{
          "Total Sentences": total_sentences,
          "Total Errors": error_count,
          "Error Rate": error_rate}

# Test
paragraph = transcrib("/content/2-3 min introduction video.mp4")
text = check_paragraph_grammar(paragraph)


Error found in sentence: Not only do I want to be able to travel the world but I want to be able to use my education and knowledge of the English language that I have learned since I was a child to other students who want to learn.


In [8]:
import librosa
import numpy as np
from moviepy.editor import *

# Extract audio from video
video_path = "/content/2-3 min introduction video.mp4"
audio_path = "extracted_audio.wav"
video = VideoFileClip(video_path)
video.audio.write_audiofile(audio_path)

import speech_recognition as sr
import librosa
import numpy as np

def analyze_speech(audio_file):
    # Load audio
    y, sr_rate = librosa.load(audio_file, sr=None)

    # Speech recognition for transcription (optional)
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)

    try:
        text = recognizer.recognize_google(audio_data)
    except sr.UnknownValueError:
        text = ""

    # Assuming one phoneme per word for simplicity
    num_words = len(text.split())

    # Calculate total duration (including pauses)
    total_duration = librosa.get_duration(y=y, sr=sr_rate)

    # Detect silent and sentence-internal pauses using an energy threshold
    intensity = librosa.feature.rms(y=y)  # Corrected call to rms function
    avg_intensity = np.mean(intensity)
    silence_threshold = -30  # Decibels, adjust based on your audio
    silent_sections = librosa.effects.split(y, top_db=-silence_threshold, ref=np.max, frame_length=2048, hop_length=512)
    silence_durations = [(stop - start) / sr_rate for start, stop in silent_sections]
    total_silence_duration = sum(silence_durations)
    num_silent_pauses = len(silence_durations)
    mean_length_of_pauses = total_silence_duration / num_silent_pauses if num_silent_pauses else 0

    # Define the threshold for considering pauses (in seconds)
    pause_threshold = 3

    # Calculate silent sections as before
    silent_sections = librosa.effects.split(y, top_db=-silence_threshold, ref=np.max, frame_length=2048, hop_length=512)

    # Filter silent sections to include only those longer than the pause_threshold
    long_silent_sections = [section for section in silent_sections if (section[1] - section[0]) / sr_rate > pause_threshold]

    # Calculate durations for the filtered (long) silent sections
    long_silence_durations = [(stop - start) / sr_rate for start, stop in long_silent_sections]
    total_long_silence_duration = sum(long_silence_durations)

    # Calculate mean length of the filtered pauses
    mean_length_of_long_pauses = total_long_silence_duration / len(long_silence_durations) if len(long_silence_durations) > 0 else 0


    # Calculate speech metrics
    rate_of_speech = num_words / total_duration
    phonation_ratio = 100 * (total_duration - total_silence_duration) / total_duration
    articulation_rate = num_words / (total_duration - total_silence_duration)

    # Number of filled pauses
    filled_pauses = ["uh", "um", "er", "ah", "like"]
    num_filled_pauses = sum(text.count(pause) for pause in filled_pauses)

    # Number of dysfluencies (simple repetition example)
    words = text.split()
    repeated_words = [word for i, word in enumerate(words[:-1]) if word == words[i+1] and word not in filled_pauses]
    num_dysfluencies = len(repeated_words)

    # Calculate silent sections as before
    silent_sections = librosa.effects.split(y, top_db=-silence_threshold, ref=np.max, frame_length=2048, hop_length=512)
    silence_durations = [(stop - start) / sr_rate for start, stop in silent_sections]
    total_silence_duration = sum(silence_durations)

    # Assuming the audio starts with a nonsilent segment and ends with a nonsilent segment
    # Initialize the first nonsilent segment starting from 0 to the start of the first silent section
    if silent_sections.size > 0:
        nonsilent_segments = [[0, silent_sections[0][0]]]
    else:
        nonsilent_segments = [[0, len(y)]]

    # Add intervals between silent sections as nonsilent segments
    for i in range(1, len(silent_sections)):
        start = silent_sections[i-1][1]
        stop = silent_sections[i][0]
        nonsilent_segments.append([start, stop])

    # Add the last nonsilent segment from the end of the last silent section to the end of the audio
    if silent_sections.size > 0:
        nonsilent_segments.append([silent_sections[-1][1], len(y)])

    # Calculate durations for nonsilent (speech) segments
    speech_durations = [(stop - start) / sr_rate for start, stop in nonsilent_segments]
    mean_length_of_runs = sum(speech_durations) / len(speech_durations) if len(speech_durations) > 0 else 0


    return {
        "Rate of Speech (words/second)": rate_of_speech,
        "Phonation Time Ratio (%)": phonation_ratio,
        "Articulation Rate (words/second of speaking)": articulation_rate,
        "Number of Silent Pauses": num_silent_pauses,
        "Mean Length of Pauses (seconds)": mean_length_of_pauses,
        "Mean Long of Pauses (seconds)":mean_length_of_long_pauses,
        "Mean Length of Runs (seconds)": mean_length_of_runs,
        "Number of Filled Pauses": num_filled_pauses,
        "Number of Dysfluencies": num_dysfluencies,
    }

# Example usage
audio_file = audio_path  # Replace with the path to your audio file
speech_features = analyze_speech(audio_file)
for feature, value in speech_features.items():
    print(f"{feature}: {value}")



MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.
Rate of Speech (words/second): 2.015810276679842
Phonation Time Ratio (%): 38.05870595932709
Articulation Rate (words/second of speaking): 5.296581231201385
Number of Silent Pauses: 181
Mean Length of Pauses (seconds): 0.4329046240964156
Mean Long of Pauses (seconds): 1.2754389374797537
Mean Length of Runs (seconds): 0.2645289177942238
Number of Filled Pauses: 19
Number of Dysfluencies: 0


In [ ]:
import pandas as pd
voice_data_1 = pd.DataFrame([speech_features])
voice_data_1

,Rate of Speech (words/second),Phonation Time Ratio (%),Articulation Rate (words/second of speaking),Number of Silent Pauses,Mean Length of Pauses (seconds),Mean Length of Runs (seconds),Number of Filled Pauses,Number of Dysfluencies
0,2.01581,58.534484,3.4438,272,0.192845,0.192845,19,0


In [ ]:
text_data_1=pd.DataFrame([text])
text_data_1

,Total Sentences,Total Errors,Error Rate
0,22,1,4.545455
